In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
directory = 'data'
all_data = []
for filename in os.listdir(directory):
    with open(os.path.join(directory, filename), 'r') as f:
        all_data.append(f.name)

all_data

['data\\merge.csv',
 'data\\merged',
 'data\\merged.csv',
 'data\\songs-1970.csv',
 'data\\songs-1971.csv',
 'data\\songs-1972.csv',
 'data\\songs-1973.csv',
 'data\\songs-1974.csv',
 'data\\songs-1975.csv',
 'data\\songs-1976.csv',
 'data\\songs-1977.csv',
 'data\\songs-1978.csv',
 'data\\songs-1979.csv',
 'data\\songs-1980.csv',
 'data\\songs-1981.csv',
 'data\\songs-1982.csv',
 'data\\songs-1983.csv',
 'data\\songs-1984.csv',
 'data\\songs-1985.csv',
 'data\\songs-1986.csv',
 'data\\songs-1987.csv',
 'data\\songs-1988.csv',
 'data\\songs-1989.csv',
 'data\\songs-1990.csv',
 'data\\songs-1991.csv',
 'data\\songs-1992.csv',
 'data\\songs-1993.csv',
 'data\\songs-1994.csv',
 'data\\songs-1995.csv',
 'data\\songs-1996.csv',
 'data\\songs-1998.csv',
 'data\\songs-1999.csv',
 'data\\songs-2000.csv',
 'data\\songs-2001.csv',
 'data\\songs-2002.csv',
 'data\\songs-2003.csv',
 'data\\songs-2005.csv',
 'data\\songs-2006.csv',
 'data\\songs-2007.csv',
 'data\\songs-2008.csv',
 'data\\songs-200

In [3]:
df = pd.concat(
    map(pd.read_csv, all_data), ignore_index=True)
songs_df = df.drop(['Unnamed: 0'], axis=1)

print(songs_df)
print(songs_df.columns)

                           id                        title  \
0      7iN1s7xHE4ifF5povM6A48  Let It Be - Remastered 2009   
1      6QhXQOpyYvbpdbyjgAqKdY                      Cecilia   
2      2BhrrVRC8CoiWejbzj0VjQ        Son of a Preacher Man   
3      3Bh6uInhcVBVvLraGZdkKD         Who'll Stop The Rain   
4      6UkMcAA19lTdjs22jtB7o2              Big Yellow Taxi   
...                       ...                          ...   
47995  7AODNxFNPnKaz4hURLzw5l               Ladbroke Grove   
47996  44ADyYoY5liaRa3EOAl4uf                   Slide Away   
47997  2KKijf6Ett4kmVNJze2roh                   The Git Up   
47998  2bjUEg4jBtKBlPdNrTAppI                       Easier   
47999  3cWI6Hj9LQ0MfMuhw9uSMc             Lost Without You   

                     primary_artist                       all_artists  \
0                       The Beatles                   ['The Beatles']   
1                 Simon & Garfunkel             ['Simon & Garfunkel']   
2                   Aretha Franklin 

In [4]:
with open('./data/merge.csv', 'w', encoding='utf-8') as f:
        songs_df.to_csv(f, header= True, index=False)